In [ ]:
################# LAST VERSION #####################

In [ ]:
#Whole code
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')

spark = SparkSession.builder.config(conf=conf).getOrCreate()

df = spark.read.option("inferSchema", 
"true").option("header", 
"true").csv("s3://final-for-bigdata/Bigdata_project_final_data_final.csv")

#df.printSchema()

#### data preparation for ML ##########
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['id','state','goal',	'usd_pledged','backers_count','comments_count','updates_count','project_duration'
                                               ,'country_AU','country_BE','country_CA','country_CH','country_DE','country_DK','country_ES','country_FR','country_GB','country_HK','country_IE','country_IT','country_JP','country_LU','country_MX','country_NL','country_NO','country_NZ','country_SE','country_SG','country_US'
                                               ,'spotlight_True','staff_pick_True','disable_communication_True','profile_show_feature_image_True','profile_state_inactive'
                                               ,'p_category_comics','p_category_crafts','p_category_dance','p_category_design','p_category_fashion','p_category_film & video','p_category_food','p_category_games','p_category_journalism','p_category_music','p_category_photography',	'p_category_publishing','p_category_technology','p_category_theater']
                                  , outputCol = 'features')
v_df = vectorAssembler.transform(df)
v_df = v_df.select(['features', 'state'])
v_df.show(3)

###### split data ################
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]
#####build logistic ML model ############
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='state', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
training_sum = lr_model.summary
training_sum.predictions.describe().show()

#evaluate logistic regression model with AUC
lr_prediction = lr_model.transform(test_df)
lr_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='state')
lr_auc = lr_eval.evaluate(lr_prediction)
lr_auc

#evaluate logistic regression model with accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
evaluatorMulti = MulticlassClassificationEvaluator(labelCol='state', predictionCol='prediction')
evaluator = BinaryClassificationEvaluator(labelCol='state', rawPredictionCol='prediction', metricName='areaUnderROC')

l_prediction = lr_prediction.select('state', 'prediction')

l_accuracy = evaluatorMulti.evaluate(l_prediction, {evaluatorMulti.metricName: 'accuracy'})
l_accuracy

########## random forest ################
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'state')
rf_model = rf.fit(train_df)
rf_prediction = rf_model.transform(test_df)

#evaluate random forest model with AUC
rf_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='state')
rf_auc = rf_eval.evaluate(rf_prediction)
rf_auc

#evaluate random forest model with accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
evaluatorMulti = MulticlassClassificationEvaluator(labelCol='state', predictionCol='prediction')
evaluator = BinaryClassificationEvaluator(labelCol='state', rawPredictionCol='prediction', metricName='areaUnderROC')

r_prediction = rf_prediction.select('state', 'prediction')

r_accuracy = evaluatorMulti.evaluate(r_prediction, {evaluatorMulti.metricName: 'accuracy'})
r_accuracy


######
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'state')
rf_model = rf.fit(train_df)
rf_prediction = rf_model.transform(test_df)

rf_eval = BinaryClassificationEvaluator(rawPredictionCol='features',labelCol='state')
rf_auc = rf_eval.evaluate(rf_prediction)
rf_auc



In [ ]:


## try
predictionAndTarget =lr_prediction.select("state", "features")

acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})


from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr_prediction2 = lr_prediction.withColumn("state",lr_prediction.state.cast('double'))
lr_prediction2.head()
#my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='feature', metricName='f1')
#my_mc_lr.evaluate(lr_prediction2)

########## useless ##############
trainingSummary = lr_model.summary

objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))